________

# COURSE WORK | VIDEO CONFERENCEING 

### CSS|18U|17U|4196

_______

# PROMPT 1
List the technologies involved in building a video conferencing systeming?.

## Response 

### Building a video conferencing system involves a variety of technologies and components, spanning both hardware and software. Here are the key technologies involved:

1. **Networking Technologies**
    - **Internet Protocol (IP):** The fundamental protocol for communication over the internet.
    - **Transmission Control Protocol (TCP) and User Datagram Protocol (UDP):** TCP ensures reliable data transmission, while UDP is used for real-time communications due to its lower latency.
    - **WebRTC (Web Real-Time Communication):** An open-source project that provides web applications and websites with real-time communication via simple application programming interfaces (APIs).
    - **Session Initiation Protocol (SIP):** A protocol used for initiating, maintaining, and terminating real-time sessions.
    - **Real-Time Transport Protocol (RTP):** A protocol for delivering audio and video over IP networks.

2. **Audio and Video Technologies**
    - **Audio/Video Codecs:**
        - **H.264/AVC, H.265/HEVC:** Video compression standards.
        - **VP8, VP9:** Open video compression formats.
        - **Opus:** An audio codec for interactive speech and music transmission.
        - **AAC (Advanced Audio Coding):** A popular audio codec for high-quality sound.
    - **Echo Cancellation:** Reduces echo to improve audio quality.
    - **Noise Reduction:** Minimizes background noise.


# Promt 2

write a code that  utilizes WebRTC to construct a seamless, scalable web conference, enabling users to create meetings, schedule meetings, add friends, record conferences, and share screens.

# Response 

Creating a full-fledged web conferencing application using WebRTC involves several steps, from setting up the backend to managing the frontend interfaces. Below is a simplified version of such an application using Node.js for the backend and plain JavaScript for the frontend. This example will cover basic functionalities such as creating meetings, joining meetings, and screen sharing. Note that building a production-ready system requires more robust handling of errors, security, and scalability concerns.



### Backend (Node.js with Express and Socket.io)
Install Dependencies

In [ ]:
npm install express socket.io uuid

server.js

In [ ]:
const express = require('express');
const http = require('http');
const socketIo = require('socket.io');
const { v4: uuidv4 } = require('uuid');

const app = express();
const server = http.createServer(app);
const io = socketIo(server);

app.use(express.static('public'));

app.get('/', (req, res) => {
    res.send('Welcome to the WebRTC Video Conferencing App');
});

app.get('/create-meeting', (req, res) => {
    const meetingId = uuidv4();
    res.redirect(`/meeting/${meetingId}`);
});

app.get('/meeting/:meetingId', (req, res) => {
    res.sendFile(__dirname + '/public/meeting.html');
});

io.on('connection', (socket) => {
    socket.on('join-meeting', (meetingId, userId) => {
        socket.join(meetingId);
        socket.to(meetingId).emit('user-connected', userId);

        socket.on('disconnect', () => {
            socket.to(meetingId).emit('user-disconnected', userId);
        });
    });
});

const PORT = process.env.PORT || 3000;
server.listen(PORT, () => {
    console.log(`Server is running on port ${PORT}`);
});


Frontend (HTML and JavaScript)
public/index.html

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WebRTC Video Conferencing</title>
</head>
<body>
    <h1>WebRTC Video Conferencing App</h1>
    <a href="/create-meeting">Create a Meeting</a>
</body>
</html>


- public/meeting.html

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Meeting</title>
    <style>
        video {
            width: 300px;
            height: 300px;
            margin: 10px;
        }
    </style>
</head>
<body>
    <h1>Meeting</h1>
    <div id="video-grid"></div>
    <button id="share-screen">Share Screen</button>
    <script src="/socket.io/socket.io.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/peerjs@1.3.2/dist/peerjs.min.js"></script>
    <script>
        const socket = io('/');
        const videoGrid = document.getElementById('video-grid');
        const myPeer = new Peer(undefined, {
            host: '/',
            port: '3001'
        });
        const myVideo = document.createElement('video');
        myVideo.muted = true;
        const peers = {};

        navigator.mediaDevices.getUserMedia({
            video: true,
            audio: true
        }).then(stream => {
            addVideoStream(myVideo, stream);

            myPeer.on('call', call


In [ ]:
const localVideo = document.getElementById('localVideo');
const remoteVideo = document.getElementById('remoteVideo');
const startCallButton = document.getElementById('startCall');
const shareScreenButton = document.getElementById('shareScreen');
const recordButton = document.getElementById('record');
const stopRecordingButton = document.getElementById('stopRecording');
const socket = io();



let localStream;
let remoteStream;
let peerConnection;
let mediaRecorder;
let recordedChunks = [];

// ICE servers configuration
const iceServers = {
    iceServers: [
        { urls: 'stun:stun.l.google.com:19302' }
    ]
};

// Get user media
navigator.mediaDevices.getUserMedia({ video: true, audio: true })
    .then(stream => {
        localStream = stream;
        localVideo.srcObject = stream;
    })
    .catch(error => console.error('Error accessing media devices.', error));

// Create Peer Connection
function createPeerConnection() {
    peerConnection = new RTCPeerConnection(iceServers);
    peerConnection.onicecandidate = handleICECandidateEvent;
    peerConnection.ontrack = handleTrackEvent;
    localStream.getTracks().forEach(track => peerConnection.addTrack(track, localStream));
}

// Handle ICE Candidate Event
function handleICECandidateEvent(event) {
    if (event.candidate) {
        // Send the candidate to the remote peer
        console.log('New ICE candidate', event.candidate);
    }
}

// Handle Track Event
function handleTrackEvent(event) {
    remoteVideo.srcObject = event.streams[0];
}

// Start Call
startCallButton.addEventListener('click', () => {
    createPeerConnection();
    peerConnection.createOffer()
        .then(offer => peerConnection.setLocalDescription(offer))
        .then(() => {
            // Send the offer to the remote peer
            console.log('Offer sent:', peerConnection.localDescription);
        });
});

// Share Screen
shareScreenButton.addEventListener('click', () => {
    navigator.mediaDevices.getDisplayMedia({ video: true })
        .then(screenStream => {
            let screenTrack = screenStream.getVideoTracks()[0];
            peerConnection.addTrack(screenTrack, screenStream);

            // Replace video track in local video element
            localVideo.srcObject = screenStream;

            // Listen for the screen track to stop (e.g., when the user stops sharing their screen)
            screenTrack.onended = () => {
                localStream.getTracks().forEach(track => peerConnection.addTrack(track, localStream));
                localVideo.srcObject = localStream;
            };
        })
        .catch(error => console.error('Error sharing screen.', error));
});

// Record Conference
recordButton.addEventListener('click', () => {
    recordedChunks = [];
    const options = { mimeType: 'video/webm; codecs=vp9' };
    mediaRecorder = new MediaRecorder(localStream, options);
    
    mediaRecorder.ondataavailable = handleDataAvailable;
    mediaRecorder.onstop = handleStop;
    mediaRecorder.start();
    console.log('Recording started.');
});

function handleDataAvailable(event) {
    if (event.data.size > 0) {
        recordedChunks.push(event.data);
    }
}

function handleStop(event) {
    const blob = new Blob(recordedChunks, { type: 'video/webm' });
    const url = URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.style.display = 'none';
    a.href = url;
    a.download = 'recorded_conference.webm';
    document.body.appendChild(a);
    a.click();
    URL.revokeObjectURL(url);
}

stopRecordingButton.addEventListener('click', () => {
    mediaRecorder.stop();
    console.log('Recording stopped.');
});



// Send Offer
peerConnection.createOffer()
    .then(offer => {
        peerConnection.setLocalDescription(offer);
        socket.emit('offer', offer);
    });

// Handle Offer
socket.on('offer', (offer) => {
    createPeerConnection();
    peerConnection.setRemoteDescription(new RTCSessionDescription(offer))
        .then(() => peerConnection.createAnswer())
        .then(answer => {
            peerConnection.setLocalDescription(answer);
            socket.emit('answer', answer);
        });
});

// Handle Answer
socket.on('answer', (answer) => {
    const remoteDesc = new RTCSessionDescription(answer);
    peerConnection.setRemoteDescription(remoteDesc);
});

// Handle ICE Candidate
socket.on('candidate', (candidate) => {
    const iceCandidate = new RTCIceCandidate(candidate);
    peerConnection.addIceCandidate(iceCandidate);
});

peerConnection.onicecandidate = (event) => {
    if (event.candidate) {
        socket.emit('candidate', event.candidate);
    }
};

peerConnection.ontrack = (event) => {
    remoteVideo.srcObject = event.streams[0];
};

# Conclusion
This example provides a basic implementation of a WebRTC-based video conferencing system with screen sharing and recording capabilities. For a fully functional system, additional features like user authentication, scheduling, and friend management would require further development, including back-end services, database integration, and more robust client-side handling.

## Deployed link



https://www.video_conferencing.vercel.app